In [1]:
!git clone https://github.com/LiSiruiRay/self-attention-on-stock.git

Cloning into 'self-attention-on-stock'...
remote: Enumerating objects: 6188, done.
remote: Counting objects: 100% (6188/6188), done.
remote: Compressing objects: 100% (6108/6108), done.
remote: Total 6188 (delta 86), reused 6170 (delta 68), pack-reused 0
Receiving objects: 100% (6188/6188), 19.27 MiB | 19.51 MiB/s, done.
Resolving deltas: 100% (86/86), done.


In [2]:
%cd self-attention-on-stock/
!pwd

/content/self-attention-on-stock
/content/self-attention-on-stock


In [3]:
# Author: ray
# Date: 1/23/24
# Description: runable start training
import time

import torch
from torch import nn

import sys
# Add the project root directory to Python's module search path
sys.path.append('/content/self-attention-on-stock')

In [4]:
from datatype.training_dataset import Mydataset
from training_pipeline import StockPredictionModel, train_model, validate_model, create_dataloaders, \
    get_cosine_schedule_with_warmup, valid, save_model, set_seed
from util.common import get_now_time_with_time_zone

Hash Code: d6584ebc7b834c41be20f958e036f1a1


In [5]:
seed = 78
set_seed(seed=seed)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print(f"[Info]: Use {device} now!")



[Info]: Use cuda now!


In [6]:
use_reduced_passage_vec = False

num_epochs = 5  # Set the number of epochs

time_features = 3

passage_vec_size = 128 if use_reduced_passage_vec else 768
output_size = 6

d_model = 2048

num_training_steps = 5000

batch_size = 2000
nhead = 8
transformer_encoder_layer_num = 20

In [7]:
model = StockPredictionModel(passage_vec_size=passage_vec_size,
                             time_features=time_features,
                             d_model=d_model,
                             output_size=output_size,
                             nhead=nhead,
                             transformer_encoder_layer_num=transformer_encoder_layer_num)

model = model.to(device).float()

mds = Mydataset(use_reduced_passage_vec=use_reduced_passage_vec)

train_loader, test_loader = create_dataloaders(dataset=mds,
                                               batch_size=batch_size)

criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=100, num_training_steps=num_training_steps)
print(f"[Info]: Finish creating model!", flush=True)

/usr/local/lib/python3.10/dist-packages/torch/nn/modules/transformer.py:282: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


[Info]: Finish creating model!


In [ ]:
from tqdm.notebook import tqdm
# pbar = tqdm(total=num_training_steps, ncols=0, desc="Train", unit=" step")

training_start_time = time.time()
train_model(model=model,
            train_loader=train_loader,
            criterion=criterion,
            optimizer=optimizer,
            scheduler=scheduler,
            num_epochs=num_epochs,
            device=device,
            num_training_steps=num_training_steps,
            pbar=None)
valid(model=model, dataloader=test_loader, criterion=criterion, device=device)
training_end_time = time.time()
training_duration = training_end_time - training_start_time

Epoch [1/5]


Train:  24% 1201/5000 [41:56<2:10:37,  2.06s/ step, loss=0.10, step=1201]

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:

description = "more layer training, this spent more GPU RAM (11 / 15)"

now_time_str = get_now_time_with_time_zone()

training_machine = "Google Colab 15GB T4 GPU"

model_info = {
    "passage vector size": passage_vec_size,
    "num of epochs": num_epochs,
    "d_model": d_model,
    "description": description,
    "time started training": now_time_str,
    "training step num": num_training_steps,
    "batch size": batch_size,
    "seed": seed,
    "nhead": nhead,
    "transformer encoder layer num": transformer_encoder_layer_num,
    "training time (sec)": training_duration,
    "training machine": training_machine,
}

save_model(model=model,
           info=model_info)